# **Implementing a Matrix Factorization-based Recommender System**

## **Represent user and item by Matrix Factorization**
 - Users and items are represented through matrix factorization.  - A user-item interaction matrix$( R \in \mathbb{R}^{n \times m}$) is approximated as the product of two matrices:$( R \approx P \times Q$), where$( P \in \mathbb{R}^{n \times d}$) and$( Q \in \mathbb{R}^{m \times d}$).
  - $ n $ is the number of users, $ m $ is the number of items, and $ d $ is the dimension of the embedding vectors.

**How to do Matrix Factorization**:
   - The goal is to find a good representation for users and items.
   - The objective is to minimize the differences between the predicted and actual interaction values: $ \min_{P,Q} \sum_{(u,i) \in R'} (r_{ui} - P_u Q_i)^2 $.
   - Not all elements in $ R $ are known; $ R' $ is the set of known elements in $ R $.
   - $ r_{ui} $ is the interaction record of user $ u $ and item $ i $.
   - $ P_u $ is the embedding vector for user $ u $, and $ Q_i $ is the embedding vector for item $ i $.
   - The interaction probability between user $ u $ and item $ i $ is $ r_{ui} = P_u Q_i $.

## **Requirements:**
In this practice, you will implement a recommender system using **Matrix Factorization**.
You should:
   - Construct a matrix factorization-based recommender system using the positive data `train_pos.npy` provided in project 3.
   - For each user-item pair $ u, i $ in `train_pos.npy`, $ R_{ui} = 1 $.
   - If a user-item pair $ u^*, i^* $ is not in `train_pos.npy`, $ R_{u^*i^*} = 0 $.
   - The task is to find a good embedding representation for each user and item.


## **Reference Workflow**:
   1. Load the data and construct an interaction matrix.
   2. Obtain the embedding representation for each user and item.
      - **Use the objective function above and optimize the embeddings via gradient descent.**
      - **Note: The number of negative samples is much larger than that of positive samples. You can sample some negative samples in each iteration instead of using all negative samples.**
   3. Validate the effectiveness of the model.

### **Deadline:** 22:00, Dec. 20th

The practice will be checked in this lab class or the next lab class (before **Dec. 20th**) by teachers or SAs.

### **Grading:**
* Submissions in this lab class: 1.1 points.
* Submissions on time: 1 point.
* Late submissions within 2 weeks after the deadline: 0.8 points.

## **1 Load and Explore the Dataset**

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
import random

# Load the dataset
train_pos = np.load("train_pos.npy")  # Contains user-item pairs
train_neg = np.load("train_neg.npy")
data_set = np.concatenate((train_pos, train_neg), axis=0)

train_set, test_set = train_test_split(data_set, test_size=0.2)

users, items = set(train_set[:, 0]), set(train_set[:, 1])
len(train_set),train_set[:5],train_set[-5:]

(40540,
 array([[3078,  480,    0],
        [4097, 2045,    1],
        [2245, 1214,    1],
        [5984,   83,    1],
        [ 835, 1528,    1]]),
 array([[5712,  756,    1],
        [4540,  624,    1],
        [3012, 1655,    1],
        [5172,  623,    1],
        [4890,   73,    0]]))

In [10]:
n_user, n_item = max(users) + 1, max(items) + 1
n_user, n_item

(np.int64(6015), np.int64(2347))

## **2. Initialize Parameters**

Initialize the embedding matrices $P$ for users and $Q$ for items. These matrices represent the user and item embeddings.

**Fill in the missing parts:**


In [11]:
# Define the embedding dimension
dim = 60 # Specify the embedding dimension

# Initialize user and item embeddings with random values
P = np.random.normal(0, 0.1, (n_user, dim)) # Fill in the correct dimensions for user embeddings
Q = np.random.normal(0, 0.1, (n_item, dim)) # Fill in the correct dimensions for item embeddings

## **3. Optimize the embeddings via gradient descent**

The loss function to optimize is Mean Squared Error (MSE):
$$
\text{Loss} = \sum_{(u, i) \in R'} (r_{ui} - P_u Q_i^T)^2
$$

OR add the regularization term:

$$
\text{Loss} = \sum_{(u, i) \in R'} (r_{ui} - P_u Q_i^T)^2 + \lambda (\|P_u\|^2 + \|Q_i\|^2)
$$

Here:
- $ 𝑅' $ is the set of the known elements in the $ 𝑅 $
- $ r_{ui} $ is 1 for positive samples and 0 for negative samples.
- $ \lambda $ is the regularization term to prevent overfitting.


In [14]:
# TODO:
learning_rate = 1e-2
num_epochs = 50
reg = 0.1

for epoch in range(num_epochs):
    total_loss = 0
    for user, item, rating in train_set:
        pred = np.dot(P[user], Q[item])
        error = rating - pred
        grad_P = -2 * error * Q[item] + 2 * reg * P[user]
        grad_Q = -2 * error * P[user] + 2 * reg * Q[item]
        P[user] -= learning_rate * grad_P
        Q[item] -= learning_rate * grad_Q
        total_loss += error ** 2
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss}")

Epoch 1/50, Loss: 19432.895551602112
Epoch 2/50, Loss: 18845.574380311184
Epoch 3/50, Loss: 18219.315828391114
Epoch 4/50, Loss: 17524.044926835617
Epoch 5/50, Loss: 16734.53790339138
Epoch 6/50, Loss: 15836.13013532999
Epoch 7/50, Loss: 14833.060667848247
Epoch 8/50, Loss: 13753.85477155113
Epoch 9/50, Loss: 12646.769910965395
Epoch 10/50, Loss: 11565.074070008994
Epoch 11/50, Loss: 10551.401357834862
Epoch 12/50, Loss: 9630.342939875103
Epoch 13/50, Loss: 8809.904508614358
Epoch 14/50, Loss: 8086.9857785959375
Epoch 15/50, Loss: 7452.770637519291
Epoch 16/50, Loss: 6896.466122267638
Epoch 17/50, Loss: 6407.389427941055
Epoch 18/50, Loss: 5975.903529751986
Epoch 19/50, Loss: 5593.697545715765
Epoch 20/50, Loss: 5253.75389338209
Epoch 21/50, Loss: 4950.19539554227
Epoch 22/50, Loss: 4678.105838934916
Epoch 23/50, Loss: 4433.362684654174
Epoch 24/50, Loss: 4212.494280012529
Epoch 25/50, Loss: 4012.562641089311
Epoch 26/50, Loss: 3831.068761654539
Epoch 27/50, Loss: 3665.8764355294466
Ep

## **4 Verification**

Choose an appropriate metric to evaluate the results.

In [16]:
# TODO:
def calculate_mse(data, P, Q):
    mse = 0
    for user, item, rating in data:
        pred = np.dot(P[user], Q[item])
        mse += (rating - pred) ** 2
    return mse / len(data)

# Calculate MSE for the training data
train_mse = calculate_mse(test_set, P, Q)
print(f"MSE: {train_mse}")

def predict(user, item):
    return np.dot(P[user], Q[item])

accurate = 0
for user, item, rating in test_set:
    result = predict(user, item)
    accurate += int(abs(result - rating) < 0.5)

print(f"Accuracy: {accurate / len(test_set)}")

MSE: 0.2783396620906268
Accuracy: 0.635027133695116
